In [1]:
import requests
import urllib3
import pandas as pd

urllib3.disable_warnings()

In [2]:
# API strava's URL
auth_url = 'https://www.strava.com/oauth/token'

# connection to the API of Strava
def connect_strava_api(url):
    
    # auth params
    payload = {
        'client_id':'123439',
        'client_secret':'ca30493ae0006807b0b09e74abdab5bc687c998e',
        'refresh_token':'28c2b6e6864b179b20bc9f4f3fa8dbe1150df56f',
        'grant_type':'refresh_token',
        'f':'json'
    }
    
    print('Requesting the token...\n')
    
    # request the new access token through refresh_token
    res = requests.post(auth_url, data=payload, verify=False)
    access_token = res.json()['access_token']
    
    print('Access token: {}\n'.format(access_token))
    
    return access_token
access_token = connect_strava_api(auth_url)

Requesting the token...

Access token: c8be00ea644c533557826b5b100b04ffbd82db88



In [3]:
# url to request data of activites logged athelete 
activities_url = 'https://www.strava.com/api/v3/athlete/activities'

def get_strava_activities(data_url):
        
    request_page_num = 1
    all_activities = []
    
    # get all pages to be sure to have all activities
    while True:
        headers = {'Authorization': 'Bearer ' + access_token}
        params = {'per_page': 200, 'page': request_page_num}

        my_dataset = requests.get(data_url,headers = headers, params = params).json()
        
        # if activities == 0 in the page, then break the loop 
        if len(my_dataset) == 0:
            print('Activities are fully added!')
            break
            
        # add each activity to the list
        if all_activities:
            all_activities.extend(my_dataset)
        else:
            all_activities = my_dataset
        
        request_page_num += 1
    
    print('Number of activities registered: {}\n'.format(len(all_activities)))
    
    # sort activities and transform to dataframe
    sorted_activities = all_activities[::-1]
    df = pd.DataFrame(sorted_activities)
    
    display(df)
    return df

df_activities = get_strava_activities(activities_url)

Activities are fully added!
Number of activities registered: 360



,resource_state,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,...,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,average_watts,kilojoules,device_watts
0,2,"{'id': 36334506, 'resource_state': 1}",Night Run,5851.9,2072,291557,15.6,Run,Run,NaN,...,2140930086,garmin_push_3142419849,False,0,0,False,68.0,NaN,NaN,NaN
1,2,"{'id': 36334506, 'resource_state': 1}",Evening Run,6501.4,2294,2434,15.5,Run,Run,NaN,...,2140930087,garmin_push_3174660568,False,0,0,False,45.0,NaN,NaN,NaN
2,2,"{'id': 36334506, 'resource_state': 1}",Lunch Run,6627.0,2380,2396,15.9,Run,Run,NaN,...,2140930088,garmin_push_3176056246,False,1,0,False,83.0,NaN,NaN,NaN
3,2,"{'id': 36334506, 'resource_state': 1}",Evening Run,5000.3,1904,2025,12.8,Run,Run,NaN,...,2140930083,garmin_push_3178619560,False,4,0,False,57.0,NaN,NaN,NaN
4,2,"{'id': 36334506, 'resource_state': 1}",Lunch Run,7741.6,2422,2433,21.2,Run,Run,NaN,...,2140930084,garmin_push_3188644460,False,7,0,False,75.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,2,"{'id': 36334506, 'resource_state': 1}",Evening Workout,0.0,1684,1684,0.0,Workout,Workout,NaN,...,11977268153,garmin_ping_334435560655,False,0,0,False,3.0,NaN,NaN,NaN
356,2,"{'id': 36334506, 'resource_state': 1}",Afternoon Run,15119.0,5526,6333,262.7,Run,Run,NaN,...,11988129214,garmin_ping_334765825218,False,1,0,False,63.0,NaN,NaN,NaN
357,2,"{'id': 36334506, 'resource_state': 1}",Afternoon Run,13585.8,5587,5684,182.7,Run,Run,0.0,...,11995733682,garmin_ping_334980907639,False,0,0,False,28.0,NaN,NaN,NaN
358,2,"{'id': 36334506, 'resource_state': 1}",Evening Run,10860.0,3003,3049,0.0,Run,Run,NaN,...,12019564032,garmin_ping_335685727546,False,0,0,False,15.0,NaN,NaN,NaN


In [4]:
# filter to run activities
df_activities = df_activities[df_activities['type'] == 'Run']
print('Number of run activity: {}'. format(len(df_activities)))

# get list of ids for future requests
lst_id_activities = [activity.get('id') for activity in df_activities.to_dict(orient='records')]
print('Number of id activities: {}'.format(len(lst_id_activities)))

Number of run activity: 206
Number of id activities: 206


In [5]:
import time
import sys

# build the request link from list of ids
lst_link_zones_activities = ['https://www.strava.com/api/v3/activities/' + str(ids) + '/zones' for ids in lst_id_activities]
print('Numbers of links to treat: {}'.format(len(lst_link_zones_activities)))

Numbers of links to treat: 206


In [ ]:
# get data of each activity
def get_strava_zones_activities(data_url, ids):
    headers = {'Authorization': 'Bearer ' + access_token}
    my_dataset = requests.get(data_url, headers=headers).json()
    
    df = pd.DataFrame(my_dataset)
    
    # Add a new column for IDs
    df.insert(0, 'id', ids)
    
    return df

all_zones_activities = []
num_activities = 0

# break the loop each 90 activities b/o api constraint
for link, ids in zip(lst_link_zones_activities, lst_id_activities):
    if num_activities == 90:
        print('activities are added, waiting 16 min to continue\n')      
        num_activities = 0
        time.sleep(960)
    else:
        # append data activities to a new list
        zone_activities_df = get_strava_zones_activities(link, ids)
        all_zones_activities.append(zone_activities_df)
        
        num_activities += 1
        print('Number of iteration {0}\r'.format(num_activities), end='', flush=True)

# concat each list 
final_df = pd.concat(all_zones_activities, ignore_index=True)
print('Length of the final df: {}'.format(len(final_df)))

Number of iteration 1
Number of iteration 2
Number of iteration 3
Number of iteration 4
Number of iteration 5
Number of iteration 6
Number of iteration 7
Number of iteration 8
Number of iteration 9
Number of iteration 10
Number of iteration 11
Number of iteration 12
Number of iteration 13
Number of iteration 14
Number of iteration 15
Number of iteration 16
Number of iteration 17
Number of iteration 18
Number of iteration 19
Number of iteration 20
Number of iteration 21
Number of iteration 22
Number of iteration 23
Number of iteration 24
Number of iteration 25
Number of iteration 26
Number of iteration 27
Number of iteration 28
Number of iteration 29
Number of iteration 30
Number of iteration 31
Number of iteration 32
Number of iteration 33
Number of iteration 34
Number of iteration 35
Number of iteration 36
Number of iteration 37
Number of iteration 38
Number of iteration 39
Number of iteration 40
Number of iteration 41
Number of iteration 42
Number of iteration 43
Number of iteration 

In [ ]:
# filter on actionable data
final_df = final_df[final_df['type'] == 'heartrate']
print('Length of the actionable df: {}'.format(len(final_df)))

In [ ]:
final_df.iloc[1]

In [ ]:
lst_zones = []

# get zones of intensity from final_df
for x in range(len(final_df)):
     zones = final_df.iloc[x].iloc[2]
     lst_zones.append(zones)

# rename columns
df_zones = pd.DataFrame(lst_zones)
df_zones.rename(columns={0: 'Zone 1',1: 'Zone 2',2: 'Zone 3',3: 'Zone 4',4: 'Zone 5'}, inplace=True)

# get only time by zones for each zones
for x in range(len(df_zones)):
    for j in range(1, len(df_zones.columns) + 1):
        df_zones.loc[x, 'Zone ' + str(j)] = df_zones.loc[x, 'Zone ' + str(j)]['time']

# transform id list in df
df_id = pd.DataFrame({'id':final_df['id']}).reset_index()

# concat id values with df_zones
result = pd.concat([df_id,df_zones], axis=1)
result.drop(columns=['index'], inplace=True)
result

In [ ]:
# merge zones_df with final_df
full_df = pd.merge(result, df_activities, on='id', how='left')
full_df

In [ ]:
# count na values
na_values = full_df.isna().sum()
na_values

In [ ]:
# count null values
null_values = full_df.isnull().sum()
null_values

In [ ]:
full_df.describe()

In [ ]:
import folium
import polyline
from gpxplotter import read_gpx_file, create_folium_map, add_segment_to_map
from folium import plugins

# create strava map
map_strava = folium.Map(location=[16.033612195704787,-61.72271466476438], zoom_start=11)

# open and read gpx file
gpx_file = read_gpx_file('TDC 2024 - Grand parcours.gpx')
df_gpx = pd.DataFrame(gpx_file)

# get latitude and longitude from df
latlon_gpx = df_gpx['segments'][0][0]['latlon']
list_latlon = [[lat, lon] for lat, lon in latlon_gpx]

# add coordinates to the map
folium.plugins.PolyLineOffset(locations=list_latlon, color='#0D19FF').add_to(map_strava)

# get Polyline from df and plot the route on a map 
def show_strava_map(df):
        
    polylines_activities = []
    
    # get all polylines from df
    for num in range(len(df)):
        df_polyline = df['map'].iloc[num]
        polylines_activities.append(df_polyline['summary_polyline'])

    clean_polyline = [ele for ele in polylines_activities if ele != '']
    
    activities_coords = []
    
    # decode each polyline to get coordinates
    for num in range(len(clean_polyline)):
        coord = polyline.decode(clean_polyline[num], 5)
        activities_coords.append(coord)
    
    # plot coords on a map
    for num in range(len(activities_coords)):
        folium.PolyLine(
            locations=activities_coords[num],
            color="#FF0000",
            weight=3).add_to(map_strava)
    
    return map_strava

show_strava_map(full_df)


In [ ]:
full_df['sport_type'] = full_df['sport_type'].replace('TrailRun', 'Run')
sport_counts = full_df['sport_type'].value_counts()

sport_counts

In [ ]:
#groupby_distance = dataframe_group_run.pivot_table(index=['rework_distance','start_date'], values='average_heartrate')
#groupby_distance.sort_values(by='average_heartrate', ascending=True)

#with pd.option_context('display.max_rows', None,
 #                      'display.max_columns', None,
  #                     'display.precision', 3):
   # display(groupby_distance)

In [ ]:
col_to_keep = ['id', 'Zone 1','Zone 2','Zone 3','Zone 4','Zone 5', 'distance', 'moving_time', 'total_elevation_gain', 'sport_type', 'start_date', 'average_speed', 'average_heartrate', 'suffer_score']

df_analyse_1 = full_df.copy()

df_analyse_1 = df_analyse_1[col_to_keep]
df_analyse_1['activity_num'] =  ['Run' + ' ' + str(index) for index in df_analyse_1.index]
df_analyse_1 ['distance'] = df_analyse_1['distance'].round(1)
df_analyse_1['moving_time'] = (df_analyse_1['moving_time'] / 60)
df_analyse_1['distance + d+'] = df_analyse_1['distance'] + (df_analyse_1['total_elevation_gain'] * 10)
df_analyse_1['start_date'] = pd.to_datetime(df_analyse_1['start_date'])
df_analyse_1['start_date'] = df_analyse_1['start_date'].dt.strftime('%d/%m/%Y')
df_analyse_1['standardize_score'] = df_analyse_1['suffer_score'] / df_analyse_1['distance + d+'] * 100

df_analyse_1

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.scatter(df_analyse_1['distance'], df_analyse_1['average_heartrate'], c=df_analyse_1['suffer_score'], s=df_analyse_1['total_elevation_gain']*2)
plt.show()

In [ ]:
df_low_intensity = df_analyse_1[df_analyse_1['Zone 2'] / (df_analyse_1['Zone 1'] + df_analyse_1['Zone 2'] + df_analyse_1['Zone 3'] + df_analyse_1['Zone 4'] + df_analyse_1['Zone 5']) > 0.80]

df_low_intensity = df_low_intensity.copy()
df_low_intensity['ratio zone 2'] = (df_analyse_1['Zone 1'] + df_analyse_1['Zone 2']) / (df_analyse_1['Zone 1'] + df_analyse_1['Zone 2'] + df_analyse_1['Zone 3'] + df_analyse_1['Zone 4'] + df_analyse_1['Zone 5'])
df_low_intensity

In [ ]:
df_low_intensity['Month_YY'] = pd.to_datetime(df_low_intensity['start_date']).dt.strftime('%Y%m')
df_low_intensity


In [ ]:
df_intensity = df_analyse_1.copy()

col_to_keep = ['Zone 1', 'Zone 2', 'Zone 3', 'Zone 4', 'Zone 5', 'distance', 'moving_time', 'average_speed', 'id','start_date']

df_intensity = df_intensity[col_to_keep]

df_intensity['Zone 2'] = df_intensity['Zone 2'] * 2
df_intensity['Zone 3'] = df_intensity['Zone 3'] * 2.5
df_intensity['Zone 4'] = df_intensity['Zone 4'] * 4
df_intensity['Zone 5'] = df_intensity['Zone 5'] * 5
df_intensity['moving_time'] = df_intensity['Zone 1'] + df_intensity['Zone 2'] + df_intensity['Zone 3'] + df_intensity['Zone 4'] + df_intensity['Zone 5']
df_intensity['start_date'] = pd.to_datetime(df_intensity['start_date'], format='%d/%m/%Y').dt.strftime('%Y/%m')
df_intensity['distance ~ intensity'] = (df_intensity['moving_time']) * df_intensity['average_speed']
df_intensity

In [ ]:
grouped_by_date = df_intensity.groupby(df_intensity['start_date'])['distance ~ intensity'].sum().sort_values(ascending=True)
grouped_sorted = grouped_by_date.sort_index()

grouped_sorted.plot(kind='bar', figsize=(10, 6))
plt.xlabel('Date')
plt.ylabel('Somme de la distance ~ intensité')
plt.title('Histogramme distance ~ intensité par date')
plt.xticks(rotation=90)  
plt.grid(axis='y')
plt.show()